In [235]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import model_selection
from sklearn import model_selection
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [236]:
#!pip install openpyxl
#!pip install scikit-learn


In [237]:
#data = pd.read_excel("yahoo_data.xlsx")
data = pd.read_csv("stocks/A.csv")
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date').reset_index(drop=True)
data.sample(5)

,Date,Open,High,Low,Close,Adj Close,Volume
4752,2018-10-09,69.339996,69.629997,68.709999,69.410004,68.478485,2179200
747,2002-11-11,9.856938,10.007154,9.656652,9.685265,8.329768,1644300
551,2002-02-01,21.530758,21.616594,20.865522,21.180258,18.215977,1535900
816,2003-02-21,9.263233,9.799714,9.041488,9.620887,8.274401,6380800
3298,2012-12-31,28.390558,29.306152,28.354792,29.284693,27.105318,4707900


In [238]:
# Inside day target and extra features to improve model accuracy
data['Inside Day'] = (data['High'] < data['High'].shift(1)) & (data['Low'] > data['Low'].shift(1))
data['Inside Day Final'] = data['Inside Day'].replace({True:1, False:0})
data['Inside Day Final'].value_counts()
data['range'] = data['High'] - data['Low']
data['prev_range'] = data['range'].shift(1)
data['compression'] = data['range'] / data['prev_range']
data['range_shrink'] = (data['range'].shift(-1) < data['range']).astype(int)
data['pct_change'] = data['Close'].pct_change()
data['volatility'] = data['range'] / data['Close']
data['prev_volatility'] = data['volatility'].shift(1)
data['volatility_ratio'] = data['volatility'] / data['prev_volatility']
data = data.dropna().reset_index(drop=True)

C:\Users\rsalt\AppData\Local\Temp\ipykernel_31900\871452800.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Inside Day Final'] = data['Inside Day'].replace({True:1, False:0})


In [239]:
print(data['Inside Day Final'].value_counts())

Inside Day Final
0    4501
1     622
Name: count, dtype: int64


In [240]:
features = data[['Open','High','Low','Close','Adj Close','Volume',
            'range','prev_range','compression','pct_change','volatility','volatility_ratio']].fillna(0).values
target = data["Inside Day Final"].values

features = np.where(np.isfinite(features), features, np.nan)
features = np.nan_to_num(features, nan=0.0, posinf=0.0, neginf=0.0)

In [241]:
# Get the split indexes
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in split.split(features, target):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = target[train_idx], target[test_idx]

In [242]:
# Scale training data ONLY
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


In [243]:
# load separate dataset to predict

df = pd.read_excel("yahoo_data.xlsx")
#df = pd.read_csv("stocks/A.csv")
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

df['Inside Day'] = (df['High'] < df['High'].shift(1)) & (df['Low'] > df['Low'].shift(1))
df['Inside Day Final'] = df['Inside Day'].replace({True:1, False:0})
df['Inside Day Final'].value_counts()
df['range'] = df['High'] - df['Low']
df['prev_range'] = df['range'].shift(1)
df['compression'] = df['range'] / df['prev_range']
df['range_shrink'] = (df['range'].shift(-1) < df['range']).astype(int)
df['pct_change'] = df['Close*'].pct_change()
df['volatility'] = df['range'] / df['Close*']
df['prev_volatility'] = df['volatility'].shift(1)
df['volatility_ratio'] = df['volatility'] / df['prev_volatility']
df = df.dropna().reset_index(drop=True)

features_new = df[['Open','High','Low','Close*','Adj Close**','Volume',
            'range','prev_range','compression','pct_change','volatility','volatility_ratio']].fillna(0).values
target_new = df["Inside Day Final"].values

features_new = np.where(np.isfinite(features_new), features_new, np.nan)
features_new = np.nan_to_num(features_new, nan=0.0, posinf=0.0, neginf=0.0)

X_test_scaled_new  = scaler.transform(features_new)


C:\Users\rsalt\AppData\Local\Temp\ipykernel_31900\3900656725.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Inside Day Final'] = df['Inside Day'].replace({True:1, False:0})


In [244]:
# creates duplicate data so model has time to learn
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_scaled, y_train)

random_forest_clf = RandomForestClassifier(random_state=42)
random_forest_clf = random_forest_clf.fit(X_train_resampled, y_train_resampled)
random_forest_test_predictions = random_forest_clf.predict(X_test_scaled)
random_forest_test_predictions_new = random_forest_clf.predict(X_test_scaled_new)

In [245]:
conf_matrix = confusion_matrix(y_test,random_forest_test_predictions)
print(conf_matrix)

[[840  61]
 [ 77  47]]


In [246]:
print("Accuracy:", accuracy_score(y_test, random_forest_test_predictions))
print(classification_report(y_test, random_forest_test_predictions))

Accuracy: 0.8653658536585366
              precision    recall  f1-score   support

           0       0.92      0.93      0.92       901
           1       0.44      0.38      0.41       124

    accuracy                           0.87      1025
   macro avg       0.68      0.66      0.66      1025
weighted avg       0.86      0.87      0.86      1025



In [247]:
conf_matrix = confusion_matrix(target_new,random_forest_test_predictions_new)
print(conf_matrix)
print("Accuracy:", accuracy_score(target_new, random_forest_test_predictions_new))
print(classification_report(target_new, random_forest_test_predictions_new))

[[1047   62]
 [ 123   25]]
Accuracy: 0.8528241845664281
              precision    recall  f1-score   support

           0       0.89      0.94      0.92      1109
           1       0.29      0.17      0.21       148

    accuracy                           0.85      1257
   macro avg       0.59      0.56      0.57      1257
weighted avg       0.82      0.85      0.84      1257

